# Coordinate Frames

Here we review some of the coordinate frames that we use when visualizing data on the globe.

We will look at three general categories of coordinate frames.

1. **Local coordinates** which keep track of the position of something relative to another point. These include East North Up (ENU) and Azimuth Elevation Range (AER).
2. **Global coordinates** which include **geodetic coordinates** and **geocentric coordinates**. Geodetic coordinates keep track of the position of something relative to a reference ellipsoid. Latitude Longitude Altitude (LLA) is a geodetic coordinate system. **Geocentric coordinate** keep track of the position of something in typical XYZ coordinates with respect to the Earth's center of mass.

We will also look briefly at standards for orienting data on the Earth, such as the World Geodetic System (WGS) and European Petroleum Survey Group (EPSG) Geodetic Parameter Dataset, which is a collection of enumerated standards, including WGS.


Let's begin with some imports.

In [1]:
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn

gv.extension("bokeh")
hv.extension("bokeh")
pn.extension()

import geoviews as gv
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn

gv.extension("bokeh")
hv.extension("bokeh")
pn.extension()

## `pymap3d`

We will use `pymap3d` to convert between the coordinate frames we encounter. You can install it with pip.

```bash
pip install pymap3d
```

We choose `pymap3d` over `pyproj` because `pymap3d` makes it easier to handle local coordinate frames such as ENU and AER.

In [2]:
import pymap3d as pm

Before we continue, we make out familiar PHL to LAX map.

In [3]:
airports = pd.read_csv("data/T_MASTER_CORD.csv").set_index("AIRPORT")
usa_airports = airports[
    airports["AIRPORT_COUNTRY_NAME"] == "United States"
].drop_duplicates(["DISPLAY_AIRPORT_NAME"])

phl = usa_airports.loc["PHL"]
lax = usa_airports.loc["LAX"]

# create a list of coordinates
path = [[phl.LONGITUDE, phl.LATITUDE], [lax.LONGITUDE, lax.LATITUDE]]

# pass list of points to gv.Points to create geographic points
points = gv.Points(path).opts(size=10, color="red")

# pass list of points **wrapped in a list** to create a geographic path
straight_line_path = gv.Path([path]).opts(color="red", line_width=2)

esri_world = gv.tile_sources.EsriImagery()

# overlay all of the elements with the world map
phl_lax_map = (esri_world * straight_line_path * points).opts(
    title="Straight-Line Path from PHL to LAX", height=480, width=640
)
phl_lax_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]

## Local Coordinates

### ENU Coordinates

East North Up (ENU) coordinates describe positions in a three-dimensional space relative to an arbitrary point. Given an arbitrary point in space, we can locate other points by moving some distance East, North, and Up from the first point. In some applications, East might mean moving to the right and North might mean moving forward (rather than referring to true East of North). ENU coordinates are common in sensing applications. A sensor might measure the position of objects in ENU coordinates with respect to its own position. To convert ENU to LLA or ECEF we need to know the position of the reference point.


In [4]:
# Generate some ENU data

import numpy as np

time_seconds = np.arange(1, 60, 0.5)

east_m = 5e5 * np.cos(time_seconds)
north_m = 1e5 * np.sin(time_seconds)
up_m = 1000 * np.ones_like(time_seconds)

# Use pymap3d to convert to LLA for plotting
lat0 = phl.LATITUDE
lon0 = phl.LONGITUDE
h0 = 0

lat, lon, alt = pm.enu2geodetic(east_m, north_m, up_m, lat0, lon0, h0)

# Zip to a list of coordinates for plotting
# **Be sure to zip at lon, lat to match the order that geoviews expects!**
coordinates = list(zip(lon, lat))

This handles the conversion for us. Now we can plot out data on the map. In this scenario, an air traffic control (ATC) radar is tracking an aircraft near PHL.

In [5]:
# pass list of points **wrapped in a list** to create a geographic path
atc_tracked_path = gv.Path([coordinates]).opts(color="blue", line_width=2)

# overlay all of the elements with the world map
phl_lax_map = (esri_world * straight_line_path * points * atc_tracked_path).opts(
    title="Straight-Line Path from PHL to LAX", height=480, width=640
)
phl_lax_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]
   .Path.II  :Path   [Longitude,Latitude]

Had the coordinates been specified as AER, we could have used the [AER functions](https://geospace-code.github.io/pymap3d/aer.html) for `pymap3d`'s interface [documentation](https://geospace-code.github.io/pymap3d/index.html).

## Global Coordinates

We have already been working in global coordinates, mostly in LLA.

Here we will see how we can use `pymap3d` to convert from ECEF to LLA and from LLA to ECEF.

In [14]:
# Get some LLA coordinates
import geopy.geocoders
from geopy.geocoders import Nominatim
import certifi
import ssl

ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

place_string = "Rutgers Camden"

geolocator = Nominatim(user_agent="ruc_data_viz_class")
location = geolocator.geocode(place_string)
altitude_m = 0

# Convert from LLA to ECEF
x_ecef_m, y_ecef_m, z_ecef_m = pm.geodetic2ecef(location.latitude, location.longitude, altitude_m)

print(f"ECEF coordinates of {place_string}:")
print(f"X: {x_ecef_m} m")
print(f"Y: {y_ecef_m} m")
print(f"Z: {z_ecef_m} m")

distance = np.sqrt(x_ecef_m ** 2 + y_ecef_m ** 2 + z_ecef_m ** 2)

print("Distance from the Earth center of mass:")
print(f"{distance/1000} km")

# Convert from ECEF back to LLA
lat, lon, alt = pm.ecef2geodetic(x_ecef_m, y_ecef_m, z_ecef_m)

print("Reconverted lat / lon:")
print(f"({lat}, {lon})")

print("Original lat / lon:")
print(f"({location.latitude}, {location.longitude})")


ECEF coordinates of Rutgers Camden:
X: 1257148.073867718 m
Y: -4732253.691342145 m
Z: 4073591.6705200872 m
Distance from the Earth center of mass:
6369.363812578247 km
Reconverted lat / lon:
(39.94836125000001, -75.12269863014204)
Original lat / lon:
(39.948361250000005, -75.12269863014204)


## Standards

While we won't get too deep into them in this class, it is important to know the standards that are used for ensuring data points can be consistently referenced and visualized correctly on maps.

The most important standard to know about is WGS 84. WGS 84 is used for positioning and navigation, including GPS and many other critical applications. WGS 84 defines an ellipsoidal coordinate frame centered at the earth's center of mass. Extensive measurements were performed to ensure the estimate of the Earth's center of mass was accurate within centimeters.

You can learn more about WGS-84 in detail from the [DoD report](https://apps.dtic.mil/sti/pdfs/ADA280358.pdf) on it and can find additional information on its [Wikipedia Page](https://en.wikipedia.org/wiki/World_Geodetic_System).

While WGS 84 is one of the most important global reference frames, you will encounter many others! Luckily, these are cataloged online at [EPGS.io](https://epsg.io/about) and recognized by `geopandas`. When using `geopandas` just be sure to be aware of which reference frame the active geometry is in.